# Enrichment analysis workflow for molecular QTL results

For molecular QTL analysis results we obtained we'd like to see if:

1. PIP are higher on average in certain annotation groups than in the rest of genome
2. Is there an enrichment for variables both in CS and in some annotation groups
    - Specifically, whether or not there is an enrichment in the secondary CS that we capture
    
We focus only on the results that has 1 or more CS identified.

In [1]:
%revisions -s -n 10

Revision,Author,Date,Message
24b7bf3,Gao Wang,2018-07-13,Update documentation
9531187,Gao Wang,2018-07-12,Add CS based enrichment
e722773,Gao Wang,2018-07-12,Update enrichment results
305751a,Gao Wang,2018-07-12,Add enrichment results


In [2]:
! sos run 20180712_Enrichment_Workflow.ipynb -h

usage: sos run 20180712_Enrichment_Workflow.ipynb
               [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  extract_sumstats
  zscore2bed
  get_variants
  range2var_annotation
  pip_rank_test
  cs_fisher_test

Global Workflow Options:
  --y-data . (as path)
                        Y data, the phenotype file paths
  --trait VAL (required)
                        Trait name
  --cwd  path(f'{y_data:d}/{trait}_output')

                        Specify work / output directory
  --annotation-dir /home/gaow/Documents/GIT/LargeFiles/he_lab_annotations_bed (as path)
                        Path to directory of annotation
                        files
  --single-annot . (as path)
         

## Annotation input


Annotation files are in bed format (for example: `Coding_UCSC.bed`).

```
chr1    69090   70008
chr1    367658  368597
chr1    621095  622034
......
chr9    141121357       141121553
chr9    141124188       141124276
chr9    141134069       141134172
```

There are many annotations one can use. For this workflow one should prepare a list of annotations for input, like this:

```
# my annotation
E8_TCM_T_48h
E8_TCM_D_48h
NR2F2
PGR_Demayo
DNaseI
H3K27me3
H3K4me3
H3K27ac
FAIRE
```

comment symbol `#` is allowed.

## Data input

Need to format the data to:

```
Variant_ID PIP z_score CS_ID 
```

Where `CS_ID` is 0 if variant is not in SuSiE CS, 1 if in CS 1, 2 in CS 2, etc. `Variant_ID` carries information of chrom and pos. eg, `rs10131831_chr14_20905250_G_A`

In [ ]:
[global]
# Y data, the phenotype file paths
parameter: y_data = path()
# Trait name
parameter: trait = None
# Specify work / output directory
parameter: cwd = path(f'{y_data:d}/{trait}_output')
# Path to directory of annotation files
parameter: annotation_dir = path('~/Documents/GIT/LargeFiles/he_lab_annotations_bed')
# Path to list of single annotations to use
parameter: single_annot = path() #parameter: single_annot = path("data/all_annotations.txt")
# Maximum distance to site of interest, set to eg. 100Kb or 1Mb up/downstream to start site of analysis unit
parameter: max_dist = 100000
fail_if(not y_data.is_file(), msg = 'Please provide valid ``--y-data``!')
z_score = path(f'{cwd}/{y_data:bnn}_{int(max_dist/1000)}Kb/enrichment/SuSiE_loci.sumstats.gz')
out_dir = f'{cwd}/{y_data:bnn}_{int(max_dist/1000)}Kb/enrichment/{z_score:bn}'.replace('.', '_')
try:
    single_anno = [f"{annotation_dir}/{x.split()[0]}.bed" for x in open(single_annot).readlines() if not x.startswith('#')]
except (FileNotFoundError, IsADirectoryError):
    single_anno = []

## Prepare summary statistics file

```
sos run analysis/20180712_Enrichment_Workflow.ipynb extract_sumstats \
    --y-data ~/Documents/GIT/LargeFiles/JointLCL/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF.txt.gz \
    --trait AS -j 16 
```

In [ ]:
# Extract summary stats from RDS files to plain text
[extract_sumstats_1]
input: glob.glob(f'{cwd}/{y_data:bnn}_{int(max_dist/1000)}Kb/SuSiE_CS_[1-9]/*.rds'), group_by = 1, concurrent = True
output: f'{_input:dd}/enrichment/{_input:bn}.sumstats.gz'
R: expand = '${ }'
    dat = readRDS(${_input:r})
    pip = dat$pip
    names = names(readRDS(dat$input)[[dat$idx]]$z_score)
    zscore = readRDS(dat$input)[[dat$idx]]$z_score
    cs_id = rep(0, length(pip))
    for (i in 1:length(dat$sets$cs)) {
        cs_id[dat$sets$cs[[i]]] = dat$sets$cs_index[i]
    }
    write.table(cbind(names,pip,zscore,cs_id), gzfile(${_output:r}), quote=FALSE, col.names=FALSE, row.names=FALSE, sep="\t")

# Consolidate results to one file
[extract_sumstats_2]
output: f'{cwd}/{y_data:bnn}_{int(max_dist/1000)}Kb/enrichment/SuSiE_loci.sumstats.gz'
bash: expand = True
    zcat {_input} | gzip --best > {_output}
_input.zap()

```
[GW] zcat /home/gaow/GIT/LargeFiles/JointLCL/AS_output/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF_100Kb/SuSiE_loci.sumstats.gz | wc -l
1667666
[GW] zcat /home/gaow/GIT/LargeFiles/JointLCL/AS_output/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF_100Kb/SuSiE_loci.sumstats.gz | cut -f 4 | grep 1 | wc -l
46659
[GW] zcat /home/gaow/GIT/LargeFiles/JointLCL/AS_output/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF_100Kb/SuSiE_loci.sumstats.gz | cut -f 4 | grep 2 | wc -l
2212
[GW] zcat /home/gaow/GIT/LargeFiles/JointLCL/AS_output/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF_100Kb/SuSiE_loci.sumstats.gz | cut -f 4 | grep 3 | wc -l
152
```

So we have total of 1667666 variants, 46659 in CS 1, 2212 in CS 2.

## Convert variants from summary statistics to bed format

In [ ]:
# Auxiliary step to get variant in bed format based on variant ID in z-score file
[zscore2bed_1]
parameter: in_file = path()
parameter: chr_prefix = ""
input: in_file
output: f'{_input:n}.bed.unsorted'
R: expand = "${ }", docker_image = 'gaow/atac-gwas', workdir = cwd, stdout = f'{_output:n}.stdout'
    library(readr)
    library(stringr)
    library(dplyr)
    var_file <- ${_input:r}
    out_file <- ${_output:r}

    variants <- read_tsv(var_file)
    colnames(variants) = c('variant', 'pip', 'zscore', 'cs')
    var_info <- str_split(variants$variant, "_")
    variants <- mutate(variants, chr = paste0("${chr_prefix}", sapply(var_info, function(x){x[2]})), 
                                 pos = sapply(var_info, function(x){x[3]})) %>%
                mutate(start = as.numeric(pos), stop=as.numeric(pos)  + 1) %>%
                select(chr, start, stop, variant)
    options(scipen=1000) # So that positions are always fully written out)
    write.table(variants, file=out_file, quote=FALSE, col.names=FALSE, row.names=FALSE, sep="\t")

[zscore2bed_2]
output: f'{_input:n}'
bash: expand = True, docker_image = 'gaow/atac-gwas', workdir = cwd
     sort-bed {_input} > {_output}
_input.zap()

[get_variants: provides = '{data}.bed']
output: f'{data}.bed'
sos_run('zscore2bed', in_file = f'{_output:n}.gz')

## Apply ranged based annotations

```
sos run analysis/20180712_Enrichment_Workflow.ipynb range2var_annotation \
    --y-data ~/Documents/GIT/LargeFiles/JointLCL/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF.txt.gz \
    --trait AS -j 16 \
    --single-annot data/annotation.list 
```

In [ ]:
# Get variants in data that falls in target region
[range2var_annotation_1]
depends: f'{z_score:n}.bed'
input: set(paths(single_anno)), group_by = 1, concurrent = True
output: f'{out_dir}/{_input:bn}.{z_score:bn}.bed'
bash: expand = True, docker_image = 'gaow/atac-gwas', workdir = cwd, volumes = f'{annotation_dir}:{annotation_dir}'
    bedops -e {z_score:n}.bed {_input} > {_output}

In [ ]:
# Make binary annotation file
[range2var_annotation_2]
depends: z_score
input: group_by = 1, concurrent = True
output: f'{_input:n}.gz'
R: expand = "${ }", docker_image = 'gaow/atac-gwas', workdir = cwd, stdout = f'{_output:n}.stdout'
    library(readr)
    library(dplyr)
    library(stringr)

    variant_tsv <- ${z_score:r}
    annotation_var_bed <- ${_input:r}
    annot_name <- ${_input:bnr} %>% str_replace(paste0(".",${z_score:bnr}), "")
    out_name <- ${_output:r}

    vars <- read_tsv(variant_tsv)[,1]
    annot_vars = read_tsv(annotation_var_bed, col_names=FALSE)
    names(vars) <- "SNP"
    vars <- vars %>%
            mutate(annot_d = case_when(SNP %in% annot_vars$X4 ~ 1,
                                                        TRUE ~ 0))
    names(vars)[2] <- annot_name
    write.table(vars, file=gzfile(out_name),
                col.names=TRUE, row.names=FALSE, sep="\t", quote=FALSE)

## Matched enrichment analysis via GREGOR

To properly perform enrichment analysis we want to match the control SNPs with the SNPs of interest -- that is, SNPs inside CS -- in terms of LD, distance to nearest gene and MAF. The [GREGOR](http://csg.sph.umich.edu/GREGOR/index.php/site/download) software can generate list of matched SNPs. I will use SNPs inside CS as input and expect a list of output SNPs matching these inputs.

GREGOR is release under University of Michigan license so I'll not make it into a docker image. So path to GREGOR directory is required. Also we need reference files, prepared by:

```
cat \
    GREGOR.AFR.ref.r2.greater.than.0.7.tar.gz.part.00 \
    GREGOR.AFR.ref.r2.greater.than.0.7.tar.gz.part.01 \
    > GREGOR.AFR.ref.r2.greater.than.0.7.tar.gz
tar zxvf GREGOR.AFR.ref.r2.greater.than.0.7.tar.gz
```

MD5SUM check:

```
AFR.part.0	( MD5: 9926904128dd58d6bf1ad4f1e90638af )
AFR.part.1	( MD5: c1d30aff89a584bfa8c1fa1bdc197f21 )
```

The input file can be a list of `rs` ID's. In `SuSiE_loci.sumstats.gz` we do have this information. 

```
sos run analysis/20180712_Enrichment_Workflow.ipynb gregor:1 \
    --y-data ~/Documents/GIT/LargeFiles/JointLCL/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF.txt.gz \
    --trait AS -j 16 \
    --single-annot data/annotation.list 
```

In [ ]:
[gregor_1 (make SNP index)]
input: z_score
output: f'{_input:nn}.rsid.txt', f'{_input:nn}.annotations.list'
bash: expand = '${ }'
    zcat ${_input} | cut -f 2,3 -d "_" | sed 's/_/:/g' > ${_output[0]}

with open(_output[1], 'w') as f:
    f.write('\n'.join(single_anno))

[gregor_2 (make configuration file)]
parameter: gregor_db = path('~/GIT/LargeFiles/GREGOR_DB')
parameter: pop = 'AFR'
output: f'{_input[0]:nn}.gregor.conf'
report: output = f'{_output}', expand = True
    ##############################################################################
    # CHIPSEQ ENRICHMENT CONFIGURATION FILE
    # This configuration file contains run-time configuration of
    # CHIP_SEQ ENRICHMENT
    ###############################################################################
    ## KEY ELEMENTS TO CONFIGURE : NEED TO MODIFY
    ###############################################################################
    INDEX_SNP_FILE = {_input[0]}
    BED_FILE_INDEX = {_input[1]} 
    REF_DIR = {gregor_db}
    R2THRESHOLD = 0.7 ## must be greater than 0.7
    LDWINDOWSIZE = 10000 ## must be less than 1MB; these two values define LD buddies
    OUT_DIR = {_output:nn}_gregor_output
    MIN_NEIGHBOR_NUM = 10 ## define the size of neighborhood
    BEDFILE_IS_SORTED = true  ## false, if the bed files are not sorted
    POPULATION = {pop}  ## define the population, you can specify EUR, AFR, AMR or ASN
    TOPNBEDFILES = 2 
    JOBNUMBER = 10
    ###############################################################################
    #BATCHTYPE = mosix ##  submit jobs on MOSIX
    #BATCHOPTS = -E/tmp -i -m2000 -j10,11,12,13,14,15,16,17,18,19,120,122,123,124,125 sh -c
    ###############################################################################
    #BATCHTYPE = slurm   ##  submit jobs on SLURM
    #BATCHOPTS = --partition=broadwl --account=pi-mstephens --time=0:30:0
    ###############################################################################
    BATCHTYPE = local ##  run jobs on local machine

bash: expand = True
    sed -i '/^$/d' {_output}

GREGOR is written in `perl`. Some libraries are required before one can run GREGOR:

```
sudo apt-get install libdbi-perl libswitch-perl libdbd-sqlite3-perl
```

In [ ]:
[gregor_3 (run gregor)]
parameter: gregor_path = path('/opt/GREGOR')
output: f'{_input:n}.done'
bash: expand = True
    perl {gregor_path}/script/GREGOR.pl --conf {_input} && touch {_output}

## Unmatched enrichment analysis

Test for larger PIP in annotation vs outside it.

```
sos run analysis/20180712_Enrichment_Workflow.ipynb pip_rank_test \
    --y-data ~/Documents/GIT/LargeFiles/JointLCL/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF.txt.gz \
    --trait AS -j 16 \
    --single-annot data/annotation.list 
```

In [ ]:
# Test if PIP is larger in annotations
[pip_rank_test_1]
depends: z_score
input_files = [f'{out_dir}/{value:bn}.{z_score:bn}.gz' for value in paths(single_anno)]
input: input_files, group_by = 1, concurrent = True
output: f'{_input:n}.{step_name}.csv'
R: expand = '${ }', docker_image = 'gaow/atac-gwas', workdir = cwd, stdout = f'{_output:n}.stdout'
    set.seed(1)
    library(readr)
    library(dplyr)
    variants <- read_tsv(${z_score:r}, col_names=FALSE)
    colnames(variants) = c('SNP', 'PIP', 'Z', 'CS')
    annotation <- read_tsv(${_input:r}, col_names=TRUE)
    name = colnames(annotation)[2]
    colnames(annotation) = c('SNP', 'GROUP')
    # add two random groupings
    annotation$RAND_1 = sample(annotation$GROUP)
    annotation$RAND_2 = sample(annotation$GROUP)
    variants = inner_join(variants, annotation, by = "SNP")
    if (length(unique(variants$GROUP)) == 1) {
      write(paste(name, NA, NA, NA, sep=','), file = ${_output:r})
    } else {
    test = wilcox.test(PIP ~ GROUP, data=variants,alternative='smaller')
    c_1 = wilcox.test(PIP ~ RAND_1, data=variants,alternative='smaller')
    c_2 = wilcox.test(PIP ~ RAND_2, data=variants,alternative='smaller')
    write(paste(name, test$p.value, c_1$p.value, c_2$p.value, sep=','), file = ${_output:r})
    }

# Consolidate results to one file
[pip_rank_test_2, cs_fisher_test_2]
output: f'{cwd}/{y_data:bnn}_{int(max_dist/1000)}Kb/enrichment/SuSiE_loci.sumstats.{step_name}.csv'
bash: expand = True
    cat {_input} > {_output}
_input.zap()

In [12]:
dat = read.table('/home/gaow/Documents/GIT/LargeFiles/JointLCL/AS_output/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF_100Kb/enrichment/SuSiE_loci.sumstats.pip_rank_test_2.csv', head=F, sep=',')
colnames(dat) = c('annotation', 'enrichment', 'random_1', "random_2")
dat[order(dat[,2],decreasing=T),]

,annotation,enrichment,random_1,random_2
51,Repressed_Hoffman,1.000000e+00,0.200018431,0.997593658
58,WeakEnhancer_Hoffman,1.000000e+00,0.952426089,0.135609932
1,1_GATA2-Interval-Track,9.999085e-01,0.988409207,0.894162343
16,FAIRE,9.998896e-01,0.961244410,0.600708171
5,CTCF_Hoffman,9.997672e-01,0.968442983,0.443205483
49,PromoterFlanking_Hoffman,9.940072e-01,0.003012780,0.518006358
46,PGR_Demayo,9.816617e-01,0.095803664,0.315776572
44,PGR-A_Bagchi,5.593118e-01,0.898392498,0.440537867
18,FOSL2,1.146906e-01,0.546745577,0.247711423
26,H3K27me3,6.588890e-02,0.780377120,0.341650134


Test for enrichment of annotation in CS.

```
sos run analysis/20180712_Enrichment_Workflow.ipynb cs_fisher_test \
    --y-data ~/Documents/GIT/LargeFiles/JointLCL/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF.txt.gz \
    --trait AS -j 16 \
    --single-annot data/annotation.list 
```

In [ ]:
# Test if CS is enriched with annotations
[cs_fisher_test_1]
depends: z_score
input_files = [f'{out_dir}/{value:bn}.{z_score:bn}.gz' for value in paths(single_anno)]
input: input_files, group_by = 1, concurrent = True
output: f'{_input:n}.{step_name}.csv'
R: expand = '${ }', docker_image = 'gaow/atac-gwas', workdir = cwd, stdout = f'{_output:n}.stdout'
    run_test = function(dat) {
        d1 = dat
        d1$CS[which(d1$CS>0)] = 1
        d1 = table(d1)
        test.d1 = fisher.test(d1)
        # test for non-first CS only
        d2 = dat[which(dat$CS != 1),]
        d2$CS[which(d2$CS>0)] = 1
        d2 = table(d2)
        test.d2 = fisher.test(d2)
        return(c(test.d1$p.value, test.d2$p.value, test.d1$estimate, test.d2$estimate))
    }
    set.seed(1)
    library(readr)
    library(dplyr)
    variants <- read_tsv(${z_score:r}, col_names=FALSE)
    colnames(variants) = c('SNP', 'PIP', 'Z', 'CS')
    annotation <- read_tsv(${_input:r}, col_names=TRUE)
    name = colnames(annotation)[2]
    colnames(annotation) = c('SNP', 'GROUP')
    # add two random groupings
    annotation$RAND_1 = sample(annotation$GROUP)
    annotation$RAND_2 = sample(annotation$GROUP)
    variants = inner_join(variants, annotation, by = "SNP")[,4:7]
    if (length(unique(variants$GROUP)) == 1) {
      write(paste(name, NA, NA, NA, NA, NA, NA, NA, NA, sep=','), file = ${_output:r})
    } else {
    # test against all CS
    test = run_test(variants[,c('CS', 'GROUP')])
    ctrl_1 = run_test(variants[,c('CS', 'RAND_1')])
    ctrl_2 = run_test(variants[,c('CS', 'RAND_2')])
    write(paste(name, test[1], test[3], test[2], test[4], ctrl_1[1], ctrl_1[2], ctrl_2[1], ctrl_2[2], sep=','), file = ${_output:r})
    }

In [5]:
dat = read.table('/home/gaow/Documents/GIT/LargeFiles/JointLCL/AS_output/fastqtl_qqnorm_ASintron_RNAseqGeuvadis_YangVCF_100Kb/enrichment/SuSiE_loci.sumstats.cs_fisher_test_2.csv', head=F, sep=',')
colnames(dat) = c('annotation', 'enrichment CS>0', 'odds ratio CS>0', 'enrichment CS>1', "odds ratio CS>1", "random_1 CS>0", "random_1 CS>1", "random_2 CS>0", "random_2 CS>1")
dat[order(dat[,2]),]

,annotation,enrichment CS>0,odds ratio CS>0,enrichment CS>1,odds ratio CS>1,random_1 CS>0,random_1 CS>1,random_2 CS>0,random_2 CS>1
8,DNaseI,0.000000e+00,0.7067594,4.409109e-27,0.6762155,2.376067e-01,0.4195083073,0.610155582,0.5423165668
40,Intron_UCSC,0.000000e+00,1.8606005,2.583183e-70,1.5002123,9.334659e-01,0.0008697541,0.001976813,0.8926382793
48,POLII_DeMayo,0.000000e+00,1.4307286,1.753827e-02,1.0689536,2.030460e-02,0.2986595412,0.114173788,0.6860353557
51,Repressed_Hoffman,0.000000e+00,0.4218858,9.669009e-47,0.6360341,3.379821e-02,0.7318257904,0.137088164,0.2475137279
52,SuperEnhancer_Hnisz,0.000000e+00,0.7328975,2.697503e-23,0.7863073,2.507967e-01,0.9246389984,0.615984051,0.0006885008
54,Transcribed_Hoffman,0.000000e+00,2.1880560,2.532327e-53,1.4262518,2.894718e-01,0.8032281724,0.310705315,0.3356351633
58,WeakEnhancer_Hoffman,3.589908e-257,0.5922669,5.908310e-08,0.6850443,4.725798e-06,0.1003845554,0.917694867,0.4464084510
24,H3K27ac_Hnisz,7.515750e-249,0.8483169,3.367817e-02,1.0513025,1.064421e-02,0.1457691657,0.876464491,0.4706701782
13,Enhancer_Hoffman,4.861672e-193,0.7667908,1.690437e-07,0.8030417,3.562017e-01,0.8115863669,0.137630288,0.5775080832
21,H3K27ac,6.624460e-193,1.6957818,5.007076e-10,1.6825087,8.501910e-01,0.0745709930,0.080183930,0.1820838977
